In [1]:
airlines_sample_table = "atp_external_catalog.source_data.AIRLINE_SAMPLE"

# Confirm AIRLINE_SAMPLE table is reflected in spark
spark.sql("SHOW TABLES IN atp_external_catalog.source_data").show(truncate=False)

df = spark.table(airlines_sample_table)

df.show()

+-----------+-------------------------+-----------+
|namespace  |tableName                |isTemporary|
+-----------+-------------------------+-----------+
|source_data|airline_sample           |false      |
|source_data|dbtools_execution_history|false      |
+-----------+-------------------------+-----------+



+---------------+--------------+------+----+-------------+--------------+---------------+
|      flight_id|       airline|origin|dest|    dep_delay|     arr_delay|       distance|
+---------------+--------------+------+----+-------------+--------------+---------------+
|1001.0000000000|Skynet Airways|   JFK| LAX|10.0000000000|  5.0000000000|2475.0000000000|
|1002.0000000000| Sunwind Lines|   ORD| SFO|-3.0000000000| -5.0000000000|1846.0000000000|
|1003.0000000000|       BlueJet|   ATL| SEA| 0.0000000000| 15.0000000000|2182.0000000000|
|1004.0000000000|Quantum Flyers|   DFW| MIA| 5.0000000000| 20.0000000000|1121.0000000000|
|1005.0000000000|Nebula Express|   BOS| DEN|12.0000000000|  8.0000000000|1754.0000000000|
|1006.0000000000|Skynet Airways|   SEA| ORD|-5.0000000000| -2.0000000000|1721.0000000000|
|1007.0000000000| Sunwind Lines|   MIA| ATL| 7.0000000000|  4.0000000000| 595.0000000000|
|1008.0000000000|       BlueJet|   SFO| BOS|22.0000000000| 18.0000000000|2704.0000000000|
|1009.0000

In [1]:
delta_path = "oci://aidp-demo-bucket@idb6enfdcxbl/delta/airline_sample"
df.write.format("delta").mode("overwrite").save(delta_path)

In [1]:

bronze_table = "airlines_data_catalog.bronze.airline_sample_delta"

# Create New Internal Catalog & Schema to store data
spark.sql("CREATE CATALOG IF NOT EXISTS airlines_data_catalog")
spark.sql("CREATE SCHEMA IF NOT EXISTS airlines_data_catalog.bronze")

# Drop the table if it exists, to avoid conflicts
spark.sql(f"DROP TABLE IF EXISTS {bronze_table}")
#spark.sql(f"DROP TABLE IF EXISTS airlines_catalog.bronze.airline_sample_delta")

# Create new bronze table
spark.sql(f"""
  CREATE TABLE IF NOT EXISTS {bronze_table}
  USING DELTA
  LOCATION '{delta_path}'
""")

DataFrame[]

In [1]:
## Clean data 
spark.sql(f"""
    DELETE FROM {bronze_table}
    WHERE DISTANCE IS NULL OR DISTANCE < 0
""")

DataFrame[num_affected_rows: bigint]

In [1]:
## Show versioning capabilities of delta tables 
## This will show old version of table before cleaning 
df_v0 = spark.read.format("delta").option("versionAsOf", 0).load(delta_path)
df_v0.show()


+---------------+--------------+------+----+-------------+--------------+---------------+
|      flight_id|       airline|origin|dest|    dep_delay|     arr_delay|       distance|
+---------------+--------------+------+----+-------------+--------------+---------------+
|1001.0000000000|Skynet Airways|   JFK| LAX|10.0000000000|  5.0000000000|2475.0000000000|
|1002.0000000000| Sunwind Lines|   ORD| SFO|-3.0000000000| -5.0000000000|1846.0000000000|
|1003.0000000000|       BlueJet|   ATL| SEA| 0.0000000000| 15.0000000000|2182.0000000000|
|1004.0000000000|Quantum Flyers|   DFW| MIA| 5.0000000000| 20.0000000000|1121.0000000000|
|1005.0000000000|Nebula Express|   BOS| DEN|12.0000000000|  8.0000000000|1754.0000000000|
|1006.0000000000|Skynet Airways|   SEA| ORD|-5.0000000000| -2.0000000000|1721.0000000000|
|1007.0000000000| Sunwind Lines|   MIA| ATL| 7.0000000000|  4.0000000000| 595.0000000000|
|1008.0000000000|       BlueJet|   SFO| BOS|22.0000000000| 18.0000000000|2704.0000000000|
|1009.0000

In [1]:
## Write cleaned data to silver schema 
df_clean = spark.table(bronze_table)

silver_path = "oci://aidp-demo-bucket@idb6enfdcxbl/delta/silver/airline_sample"
silver_table = "airlines_data_catalog.silver.airline_sample_delta"

# Create Silver Schema to store data
spark.sql("CREATE SCHEMA IF NOT EXISTS airlines_data_catalog.silver")

# Write cleaned DataFrame to object storage as Delta
df_clean.write.format("delta").mode("overwrite").save(silver_path)

# Remove table registration if it already exists
spark.sql(f"DROP TABLE IF EXISTS {silver_table}")
#spark.sql(f"DROP TABLE IF EXISTS airlines_catalog.silver.airline_sample_delta")

# Register cleaned data as new Silver table
spark.sql(f"""
  CREATE TABLE {silver_table}
  USING DELTA
  LOCATION '{silver_path}'
""")

# Check table to make sure it's cleaned 
spark.sql(f"SELECT * FROM {silver_table}").show()

+---------------+--------------+------+----+-------------+--------------+---------------+
|      flight_id|       airline|origin|dest|    dep_delay|     arr_delay|       distance|
+---------------+--------------+------+----+-------------+--------------+---------------+
|1001.0000000000|Skynet Airways|   JFK| LAX|10.0000000000|  5.0000000000|2475.0000000000|
|1002.0000000000| Sunwind Lines|   ORD| SFO|-3.0000000000| -5.0000000000|1846.0000000000|
|1003.0000000000|       BlueJet|   ATL| SEA| 0.0000000000| 15.0000000000|2182.0000000000|
|1004.0000000000|Quantum Flyers|   DFW| MIA| 5.0000000000| 20.0000000000|1121.0000000000|
|1005.0000000000|Nebula Express|   BOS| DEN|12.0000000000|  8.0000000000|1754.0000000000|
|1006.0000000000|Skynet Airways|   SEA| ORD|-5.0000000000| -2.0000000000|1721.0000000000|
|1007.0000000000| Sunwind Lines|   MIA| ATL| 7.0000000000|  4.0000000000| 595.0000000000|
|1008.0000000000|       BlueJet|   SFO| BOS|22.0000000000| 18.0000000000|2704.0000000000|
|1009.0000

In [1]:
# Enrich data by adding aggregates/average delays and distance 
from pyspark.sql import functions as F

df = spark.table("airlines_data_catalog.silver.airline_sample_delta")

# Calculate averages by airline
avg_df = df.groupBy("AIRLINE").agg(
    F.avg("DEP_DELAY").alias("AVG_DEP_DELAY"),
    F.avg("ARR_DELAY").alias("AVG_ARR_DELAY"),
    F.avg("DISTANCE").alias("AVG_DISTANCE")
)

# Join with the detail table
enhanced_df = df.join(avg_df, on="AIRLINE", how="left")

enhanced_df.show()

+--------------+---------------+------+----+-------------+--------------+---------------+----------------+----------------+-------------------+
|       airline|      flight_id|origin|dest|    dep_delay|     arr_delay|       distance|   AVG_DEP_DELAY|   AVG_ARR_DELAY|       AVG_DISTANCE|
+--------------+---------------+------+----+-------------+--------------+---------------+----------------+----------------+-------------------+
|Skynet Airways|1001.0000000000|   JFK| LAX|10.0000000000|  5.0000000000|2475.0000000000|6.00000000000000|3.60000000000000|1736.80000000000000|
| Sunwind Lines|1002.0000000000|   ORD| SFO|-3.0000000000| -5.0000000000|1846.0000000000|4.20000000000000|4.40000000000000|1470.40000000000000|
|       BlueJet|1003.0000000000|   ATL| SEA| 0.0000000000| 15.0000000000|2182.0000000000|3.60000000000000|6.20000000000000|1740.40000000000000|
|Quantum Flyers|1004.0000000000|   DFW| MIA| 5.0000000000| 20.0000000000|1121.0000000000|4.60000000000000|5.60000000000000|1280.80000000

In [1]:
# Add New Review Column for Sentiment Analysis 
import random

sample_reviews = [
    "The flight was on time and comfortable.",
    "Long delay and unfriendly staff.",
    "Quick boarding and smooth flight.",
    "Lost my luggage, not happy.",
    "Great service and tasty snacks."
]

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

random_review_udf = udf(lambda: random.choice(sample_reviews), StringType())
df_with_review = enhanced_df.withColumn("REVIEW", random_review_udf())
df_with_review.show()

+--------------+---------------+------+----+-------------+--------------+---------------+----------------+----------------+-------------------+--------------------+
|       airline|      flight_id|origin|dest|    dep_delay|     arr_delay|       distance|   AVG_DEP_DELAY|   AVG_ARR_DELAY|       AVG_DISTANCE|              REVIEW|
+--------------+---------------+------+----+-------------+--------------+---------------+----------------+----------------+-------------------+--------------------+
|Skynet Airways|1001.0000000000|   JFK| LAX|10.0000000000|  5.0000000000|2475.0000000000|6.00000000000000|3.60000000000000|1736.80000000000000|Great service and...|
| Sunwind Lines|1002.0000000000|   ORD| SFO|-3.0000000000| -5.0000000000|1846.0000000000|4.20000000000000|4.40000000000000|1470.40000000000000|Long delay and un...|
|       BlueJet|1003.0000000000|   ATL| SEA| 0.0000000000| 15.0000000000|2182.0000000000|3.60000000000000|6.20000000000000|1740.40000000000000|The flight was on...|
|Quantum F

In [1]:
# test model 
spark.sql("select query_model('cohere.command-latest','What is Intelligent Data Lake Service in Oracle?') as questions").show(truncate=False)

# Run Sentiment Analysis Against Review with LLM 
from pyspark.sql.functions import expr
enhanced_df = df_with_review.withColumn("SENTIMENT",\
                     expr("query_model('cohere.command-latest', concat('What is the sentiment for this review: ', review))"))\
#.show(10, False)

enhanced_df.show(10, False)

+-------------------------------------------------------------------------------------------------------------------+
|questions                                                                                                          |
+-------------------------------------------------------------------------------------------------------------------+
|The Intelligent Data Lake Service in Oracle is a cloud-based data management and analytics platform that provides a|
+-------------------------------------------------------------------------------------------------------------------+



+--------------+---------------+------+----+-------------+-------------+---------------+----------------+----------------+-------------------+---------------------------------------+--------------------------------------------------------------------------------------------------------------------------------+
|airline       |flight_id      |origin|dest|dep_delay    |arr_delay    |distance       |AVG_DEP_DELAY   |AVG_ARR_DELAY   |AVG_DISTANCE       |REVIEW                                 |SENTIMENT                                                                                                                       |
+--------------+---------------+------+----+-------------+-------------+---------------+----------------+----------------+-------------------+---------------------------------------+--------------------------------------------------------------------------------------------------------------------------------+
|Skynet Airways|1001.0000000000|JFK   |LAX |10.0000000000|5.0000

In [1]:
# Save Averaged Data to Gold Schema 

gold_path = "oci://aidp-demo-bucket@idb6enfdcxbl/delta/gold/airline_sample_avg"
gold_table = "airlines_data_catalog.gold.airline_sample_avg"

# Create Gold Schema 
spark.sql("CREATE SCHEMA IF NOT EXISTS airlines_data_catalog.gold")

enhanced_df.write.format("delta").option("mergeSchema", "true").mode("overwrite").save(gold_path)

spark.sql(f"DROP TABLE IF EXISTS {gold_table}")
spark.sql(f"DROP TABLE IF EXISTS airlines_catalog.gold.airline_sample_agg")

spark.sql(f"""
  CREATE TABLE {gold_table}
  USING DELTA
  LOCATION '{gold_path}'
""")

df_gold = spark.table(gold_table) 
df_gold.show()

+--------------+---------------+------+----+-------------+--------------+---------------+---------------------+---------------------+--------------------+--------------------+--------------------+----------------+----------------+-------------------+
|       airline|      flight_id|origin|dest|    dep_delay|     arr_delay|       distance|AVG_DEP_DELAY_AIRLINE|AVG_ARR_DELAY_AIRLINE|AVG_DISTANCE_AIRLINE|              review|           SENTIMENT|   AVG_DEP_DELAY|   AVG_ARR_DELAY|       AVG_DISTANCE|
+--------------+---------------+------+----+-------------+--------------+---------------+---------------------+---------------------+--------------------+--------------------+--------------------+----------------+----------------+-------------------+
|Skynet Airways|1001.0000000000|   JFK| LAX|10.0000000000|  5.0000000000|2475.0000000000|                 NULL|                 NULL|                NULL|Long delay and un...|The sentiment for...|6.00000000000000|3.60000000000000|1736.800000000000

In [1]:
# Create new external connection to adb with gold schema
# Write gold dataframe to newly created table 
# If AIRLINE_SAMPLE_GOLD does not exist in gold schema, create table first in ADB

# Before pushing dataframe, make sure all columns are upper case to prevent visualization issues in OAC 
for col_name in df_gold.columns:
    df_gold = df_gold.withColumnRenamed(col_name, col_name.upper())

df_gold.show()




+--------------+---------------+------+----+-------------+--------------+---------------+---------------------+---------------------+--------------------+--------------------+--------------------+----------------+----------------+-------------------+
|       AIRLINE|      FLIGHT_ID|ORIGIN|DEST|    DEP_DELAY|     ARR_DELAY|       DISTANCE|AVG_DEP_DELAY_AIRLINE|AVG_ARR_DELAY_AIRLINE|AVG_DISTANCE_AIRLINE|              REVIEW|           SENTIMENT|   AVG_DEP_DELAY|   AVG_ARR_DELAY|       AVG_DISTANCE|
+--------------+---------------+------+----+-------------+--------------+---------------+---------------------+---------------------+--------------------+--------------------+--------------------+----------------+----------------+-------------------+
|Skynet Airways|1001.0000000000|   JFK| LAX|10.0000000000|  5.0000000000|2475.0000000000|                 NULL|                 NULL|                NULL|Long delay and un...|The sentiment for...|6.00000000000000|3.60000000000000|1736.800000000000

In [1]:
from pyspark.sql.functions import col
from pyspark.sql.types import DecimalType, StringType

df_gold_typed = (
    df_gold
    .withColumn("FLIGHT_ID", col("FLIGHT_ID").cast(DecimalType(38,10)))
    .withColumn("DEP_DELAY", col("DEP_DELAY").cast(DecimalType(38,10)))
    .withColumn("ARR_DELAY", col("ARR_DELAY").cast(DecimalType(38,10)))
    .withColumn("DISTANCE", col("DISTANCE").cast(DecimalType(38,10)))
    .withColumn("AVG_DEP_DELAY", col("AVG_DEP_DELAY").cast(DecimalType(38,10)))
    .withColumn("AVG_ARR_DELAY", col("AVG_ARR_DELAY").cast(DecimalType(38,10)))
    .withColumn("AVG_DISTANCE", col("AVG_DISTANCE").cast(DecimalType(38,10)))
    .withColumn("AIRLINE", col("AIRLINE").cast(StringType()))
    .withColumn("ORIGIN", col("ORIGIN").cast(StringType()))
    .withColumn("DEST", col("DEST").cast(StringType()))
    .withColumn("REVIEW", col("REVIEW").cast(StringType()))
    .withColumn("SENTIMENT", col("SENTIMENT").cast(StringType()))
)

col_order = [
    "FLIGHT_ID", "AIRLINE", "ORIGIN", "DEST", "DEP_DELAY", "ARR_DELAY", "DISTANCE",
    "AVG_DEP_DELAY", "AVG_ARR_DELAY", "AVG_DISTANCE", "REVIEW", "SENTIMENT"
]

df_gold_typed = df_gold_typed.select(col_order)
print(df_gold_typed.printSchema())
df_gold_typed.createOrReplaceTempView("df_gold")
print(df_gold.printSchema())
#df_gold.createOrReplaceTempView("df_gold")

root
 |-- FLIGHT_ID: decimal(38,10) (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- DEP_DELAY: decimal(38,10) (nullable = true)
 |-- ARR_DELAY: decimal(38,10) (nullable = true)
 |-- DISTANCE: decimal(38,10) (nullable = true)
 |-- AVG_DEP_DELAY: decimal(38,10) (nullable = true)
 |-- AVG_ARR_DELAY: decimal(38,10) (nullable = true)
 |-- AVG_DISTANCE: decimal(38,10) (nullable = true)
 |-- REVIEW: string (nullable = true)
 |-- SENTIMENT: string (nullable = true)

None


root
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_ID: decimal(38,10) (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- DEP_DELAY: decimal(38,10) (nullable = true)
 |-- ARR_DELAY: decimal(38,10) (nullable = true)
 |-- DISTANCE: decimal(38,10) (nullable = true)
 |-- AVG_DEP_DELAY_AIRLINE: decimal(38,14) (nullable = true)
 |-- AVG_ARR_DELAY_AIRLINE: decimal(38,14) (nullable = true)
 |-- AVG_DISTANCE_AIRLINE: decimal(38,14) (nullable = true)
 |-- REVIEW: string (nullable = true)
 |-- SENTIMENT: string (nullable = true)
 |-- AVG_DEP_DELAY: decimal(38,14) (nullable = true)
 |-- AVG_ARR_DELAY: decimal(38,14) (nullable = true)
 |-- AVG_DISTANCE: decimal(38,14) (nullable = true)

None


In [1]:
# Verify external table accessibility
try:
    spark.sql("SELECT COUNT(*) FROM airlines_external_adb_gold.gold.airline_sample_gold").show()
    print("Target table is accessible.")
except Exception as e:
    print(f"Accessibility check failed: {e}")
    # Add sleep if needed
    import time
    time.sleep(10)

+--------+
|count(1)|
+--------+
|       0|
+--------+

Target table is accessible.


In [1]:
# Now perform the insert
spark.sql("INSERT INTO airlines_external_adb_gold.gold.airline_sample_gold SELECT * FROM df_gold")


opc-request-id: csid4b99d36a4828abdc4d142a8fab33/dd9532c0f03344dcabe6c1b03b5f403b/F434CC457E6C40D78EA9E416ABA0DEC7

Command ID failed with java.lang.RuntimeException: java.lang.Exception: [---------------------------------------------------------------------------, Py4JJavaError                             Traceback (most recent call last), Cell In[683], line 2
      1 # Now perform the insert
----> 2 spark.sql("INSERT INTO airlines_external_adb_gold.gold.airline_sample_gold SELECT * FROM df_gold")
, File /opt/spark/python/lib/pyspark.zip/pyspark/sql/session.py:1631, in SparkSession.sql(self, sqlQuery, args, **kwargs)
   1627         assert self._jvm is not None
   1628         litArgs = self._jvm.PythonUtils.toArray(
   1629             [_to_java_column(lit(v)) for v in (args or [])]
   1630         )
-> 1631     return DataFrame(self._jsparkSession.sql(sqlQuery, litArgs), self)
   1632 finally:
   1633     if len(kwargs) > 0:
, File /opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/ja

In [1]:
%sql
INSERT into airlines_external_adb_gold.gold.airline_sample_gold select * from df_gold

opc-request-id: csid4b99d36a4828abdc4d142a8fab33/8f0faaf5763844ccbb50595de7f0c0e6/5CE340D195504187817DA706E1A53B49

Command ID failed with java.lang.RuntimeException: java.lang.RuntimeException: com.oracle.dicom.connectivity.exception.ConnectorException: CONNECTOR_0084 - Exception while writing data. Possible cause: Unable to determine if path is a directory